## input:    länkar till lopp
## scrape:   någon uppgift från varje lopp (inkl datum, avd, häst)
## merge:    merge in ny kolumn i mer_komplett.csv
## output:   uppdaerad mer_komplett.csv

In [6]:
#!apt install chromium-chromedriver
!pip install selenium


import pandas as  pd 
import numpy as np
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

import logging
logging.basicConfig(filename='bet.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)


In [7]:

def getBana(the_driver, avd, after=False):
    hack = '2' if avd == '7' else '1'     # när jag kör direkt efter omgångens slut
    #hack = '1' if avd == '7' else '1'    # när jag kör innan?

    #print('bana avd', avd, 'after', after, 'hack', hack)
    end_string = '/div['+hack+']/div/div[1]/div/div[2]/div[2]/div[1]/span'
    if after:
        end_string = '/div['+hack + \
            ']/div[1]/div[1]/div/div[2]/div[2]/div[1]/span'

    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div[' + \
        avd+']/div/div' + end_string

    bana = the_driver.find_element_by_xpath(pth)

    return bana.text



def getNamn(the_driver, avd, i, after=False):

    end_string = '/td[1]/span[2]/div/span[1]'
    if after:
        end_string = '/td[1]/span/div/span[1]'

    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div[' + \
        avd + ']/div/div/table/tbody/tr[' + str(i) + ']' + end_string

    namn = the_driver.find_element_by_xpath(pth)
    return namn.text

def getVodds(the_driver, avd, i, after=False):
    # vo = '7'  #blev fel direkt efter omg
    vo = '4'  #

    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/table/tbody/tr['+str(i)+']/td['+vo+']'
    
    vodds = the_driver.find_element_by_xpath(pth)

    return vodds.text

def getStreck(the_driver, avd, i, after=False):
    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/table/tbody/tr['+str(i)+']/td[3]'
    streck = the_driver.find_element_by_xpath(pth)
    return streck.text

def getPris(the_driver, avd, after=False):
    hack='2' if avd=='7' else '1'

    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/div['+hack+']/div[1]/div[2]/div/span[3]'
    prisrad = the_driver.find_element_by_xpath(pth).text
    priser = prisrad.replace('Pris: ','')

    return priser.split(sep='-')[0]

### utdelning ###
def getUtd(the_driver):
    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[5]/div[3]/ol/li[1]/span[3]'
    utd7= the_driver.find_element_by_xpath(pth).text
    utd7=utd7.replace(' ','')
    utd7=utd7.replace('kr','')
    
    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[5]/div[3]/ol/li[2]/span[3]'
    utd6= the_driver.find_element_by_xpath(pth).text
    utd6=utd6.replace(' ','')
    utd6=utd6.replace('kr','')

    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[5]/div[3]/ol/li[3]/span[3]'
    utd5= the_driver.find_element_by_xpath(pth).text
    utd5=utd5.replace(' ','')
    utd5=utd5.replace('kr','')
    
    return utd7,utd6,utd5



In [8]:
def init_scraping(url):
    print('init')
    # get the webdriver
    driver_s = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')  
    print(f"get v75 omg 1")
    driver_s.get(url)
    driver_s.implicitly_wait(30) # seconds
    print('fullscreen')
    driver_s.fullscreen_window()

    import os
    input('klicka bort popup och tryck enter här')
    
    return driver_s

In [9]:
def do_scraping(url,driver_s):
    print('innan get url')
    driver_s.get(url)
    print('efter get url')
    driver_s.implicitly_wait(2) # seconds
    print('innan fullscreen')
    driver_s.fullscreen_window()
    print('efter fullscreen')
    driver_s.implicitly_wait(5) # seconds
    print('efter wait5')
    alla=[]
    dat = url.split('spel/')[1][0:10]  #gräv ut datum  
    print(dat)
    for avd in range(1,8): 
     # get lista med lopp-rader
        avd_pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+str(avd)+']/div/div/table/tbody'
        avd_tot = driver_s.find_elements_by_xpath(avd_pth)
        driver_s.implicitly_wait(0) # seconds
        lista = avd_tot[0].text.split('\n')

        bana = getBana(driver_s,str(avd))

        pris = getPris(driver_s,str(avd))
    
        ##### pris ######
        data = [dat, bana, avd, pris]
        alla.append(data)   
        logging.warning(f'{dat} {bana} avd={str(avd)}')

        if True:
            ## bryr mig bara och loppdata, inte häst just nu
            continue

        hst=0
        for i in range(len(lista)):
               
            if lista[i][:8] != 'Tillägg:':
                hst+=1
                
                if getVodds(driver_s,str(avd),i+1) != 'EJ':
                    #print('hitta namn')
                    namn = getNamn(driver_s,str(avd),i+1,after=True)

                    ##### streck ######
                    streck = getStreck(driver_s,str(avd),i+1)
                    data = [dat, bana, avd, hst, namn, streck]

    return alla


In [10]:
colData=[]
omg = pd.read_csv('vissa_lopp2.csv',header=None)
omg.columns=['url']
print(f'{omg.size} omgångar')
driver_s = init_scraping(omg.iloc[0].url)

for url in omg.url:
    print(url)
    colData.append(do_scraping(url,driver_s))
    
df=pd.DataFrame()
for omgång in colData:
    df = pd.concat([df,pd.DataFrame(omgång)])

#df.columns=['datum','bana','avd','nr','häst','streck']
df.columns=['datum','bana','avd','pris']
df.info()
 



FileNotFoundError: [Errno 2] File b'vissa_lopp2.csv' does not exist: b'vissa_lopp2.csv'

In [ ]:
df.to_csv('pris.csv')

In [ ]:
 df


In [ ]:
driver_s.quit()

# Special för strecken som skall petas in i mer_komplett

In [ ]:
mer_komplett = pd.read_csv('../spel/mer_komplett.csv')
                     
print(mer_komplett.shape)
mer_komplett.info()
#ta bort dubletter
# mer_komplett.drop_duplicates(['datum','häst'],inplace=True)
# mer_komplett.sort_values(by=['datum','bana','avd'],inplace=True)
# mer_komplett.reset_index(drop=True,inplace=True)
# print(mer_komplett.shape)
# mer_komplett.to_csv('mer_komplett.csv', index=False)
# mer_komplett.shape

In [ ]:
print(df.shape)
mer_komplett.shape


In [ ]:
df.datum.unique().size

In [ ]:
mer_komplett['streck'] = ' '
for  i in range(mer_komplett.shape[0]):
    #print(mer_komplett.iloc[i]['avd'])
    if df[(df.datum==mer_komplett.iloc[i].datum) & (df.häst==mer_komplett.iloc[i].häst)].avd.empty:
        print(i,'na')
    else:
        mer_komplett.avd.iloc[i] = int(df[(df.datum==mer_komplett.iloc[i].datum) & (df.häst==mer_komplett.iloc[i].häst)].avd)
        mer_komplett.streck.iloc[i] = df[(df.datum==mer_komplett.iloc[i].datum) & (df.häst==mer_komplett.iloc[i].häst)].streck.values[0]
    
        if i%1000 == 0:
            #print(df[(df.datum==mer_komplett.iloc[i].datum) & (df.häst==mer_komplett.iloc[i].häst)].avd.values)
            print(mer_komplett.iloc[i]['avd'])
            


In [ ]:
mer_komplett.sample(50)[['datum','bana','avd','häst','streck']]

In [ ]:
mer_komplett.to_csv('../spel/mer_komplett.csv',index=False)

# Special för prissumma - peta in i mer_komplett

In [ ]:
mer_komplett = pd.read_csv('../spel/mer_komplett.csv')
                     
print(mer_komplett.shape)
mer_komplett.info()
#ta bort dubletter
# mer_komplett.drop_duplicates(['datum','häst'],inplace=True)
# mer_komplett.sort_values(by=['datum','bana','avd'],inplace=True)
# mer_komplett.reset_index(drop=True,inplace=True)
# print(mer_komplett.shape)
# mer_komplett.to_csv('mer_komplett.csv', index=False)
# mer_komplett.shape


In [ ]:
print(df.shape)
mer_komplett.shape


In [ ]:
df.datum.unique().size

In [ ]:
mer_komplett['pris'] = ' '
for  i in range(mer_komplett.shape[0]):
    #print(mer_komplett.iloc[i]['avd'])
    #if df[(df.datum==mer_komplett.iloc[i].datum) & (df.häst==mer_komplett.iloc[i].häst)].avd.empty:
        #print(i,'na')
    #else:
    #mer_komplett.pris.iloc[i] = df[(df.datum==mer_komplett.iloc[i].datum) & (df.avd==mer_komplett.iloc[i].avd)].pris
    mer_komplett.pris.iloc[i]=df[(df.datum==mer_komplett.iloc[i].datum) & (df.avd.astype(int)==mer_komplett.iloc[i].avd)].pris.values[0]
    
    if i%1000 == 0:
        #print(df[(df.datum==mer_komplett.iloc[i].datum) & (df.häst==mer_komplett.iloc[i].häst)].avd.values)
        print(mer_komplett.iloc[i]['avd'])
        print(mer_komplett.iloc[i]['pris'])
        
            


In [ ]:
mer_komplett.sample(50)[['datum','bana','avd','häst','pris']]


In [ ]:
mer_komplett.to_csv('../spel/mer_komplett.csv',index=False)

In [ ]:
mer_komplett.datum.unique()